In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import chardet
import re
import warnings

warnings.filterwarnings("ignore", category=pd.errors.SettingWithCopyWarning)

# Carga de datos
## Partidos

In [2]:
df = pd.read_csv('tennis_atp-master/atp_matches_2023.csv')

In [3]:
df.head()

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,...,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points
0,2023-9900,United Cup,Hard,18,A,20230102,300,126203,3.0,NaN,...,62.0,47.0,15.0,12.0,9.0,9.0,9.0,3355.0,16.0,2375.0
1,2023-9900,United Cup,Hard,18,A,20230102,299,126207,NaN,NaN,...,12.0,8.0,3.0,4.0,1.0,3.0,19.0,2000.0,23.0,1865.0
2,2023-9900,United Cup,Hard,18,A,20230102,296,126203,3.0,NaN,...,62.0,51.0,7.0,12.0,2.0,2.0,9.0,3355.0,10.0,2905.0
3,2023-9900,United Cup,Hard,18,A,20230102,295,126207,NaN,NaN,...,41.0,26.0,12.0,9.0,6.0,9.0,19.0,2000.0,245.0,220.0
4,2023-9900,United Cup,Hard,18,A,20230102,292,126774,1.0,NaN,...,58.0,48.0,18.0,16.0,1.0,2.0,4.0,5550.0,16.0,2375.0


## Puntos

In [4]:
pbp = pd.read_csv('tennis_slam_pointbypoint-master/2023-wimbledon-points.csv')

In [5]:
print(f'Cantidad de partidos unicos: {len(pbp["match_id"].unique())}')

Cantidad de partidos unicos: 254


## Rallies

In [6]:
charting = pd.read_csv('tennis_MatchChartingProject-master/charting-m-points-2020s.csv', encoding= 'latin1', low_memory=False)
ptos_2010 = pd.read_csv('tennis_MatchChartingProject-master/charting-m-points-2010s.csv', encoding= 'latin1', low_memory=False)
ptos_to_2009 = pd.read_csv('tennis_MatchChartingProject-master/charting-m-points-to-2009.csv', encoding= 'latin1', low_memory=False)

In [7]:
#append de dataframes with concat
charting = pd.concat([charting, ptos_2010, ptos_to_2009])

In [8]:
charting['rallyCount'] = pd.to_numeric(charting['rallyCount'], errors='coerce')
charting['rallyCount'].mean()

4.033929229246487

In [9]:
partidos = pd.Series(charting['match_id'].unique())
jugador = 'Zverev'
partidos.str.contains(jugador).sum()

118

# Data Cleaning

Columnas con golpes por jugadores:

# Rally tokenization

In [10]:
# Quito informacion adicional que no aporta informacion relevante (tipos de golpes optativos, errores, etc)
charting['1st'] = charting['1st'].str.replace(r'[c\-=\+;\^]', '', regex = True)
charting['2nd'] = charting['2nd'].str.replace(r'[c\-=\+;\^]', '', regex = True)

# Parsing de los puntos

El objetivo es "parsear" los para dividir y contar los golpes de cada punto. Para ello, se utilizará una **expresiones regulares** que permita identificar los golpes de cada punto. 
*Información a cerca de cada tipo de golpe por los que se elegirá la expresión regular se puede encontrar en el diccionario de golpes: "MatchChart"*
 Las expresiones regulares funcionan como **"escudo" ante los errores de data entry**, ya que estos no son tenidos en cuenta en el parsing. Se vuelve un paso importante para la limpieza de datos, porque posiblemente haya varios errores en este sentido.

In [11]:
import pandas as pd
import re

errores = r'[ndxwg!e+]' 
errores_saque = r'[nwdxg!eVc]'

# Parsing function (customize as needed)
def parse_sequence(sequence):
    result = []
    pos = 0  # Initialize position

    # Saque
    serve_pattern = rf'^([456]{errores_saque}?)'
    match = re.match(serve_pattern, sequence)
    if match:
        serve = match.group(1)
        result.append(serve)
        pos = match.end()

    # Devolucion
    shot1_pattern = fr'([a-zA-Z]\d[789]?{errores}?)'
    match = re.match(shot1_pattern, sequence[pos:])
    if match:
        shot1 = match.group(1)
        result.append(shot1)
        pos += match.end()

    # Post devolucion
    shots_pattern = rf'([a-zA-Z]\d{errores}?)'
    shots_matches = re.finditer(shots_pattern, sequence[pos:])
    for m in shots_matches:
        result.append(m.group(1))
    pos += m.end() if 'm' in locals() else 0

    # final del punto
    error = sequence[pos:]
    if error:
        result.append(error)

    #count the number of shots
    rallyCount = len(result) - 1

    return f'{rallyCount}: ' + ' '.join(result)

In [12]:
# aplico la funcion a las columnas 1st y 2nd, que contiene la secuencia de golpes de cada punto
charting['1st_test'] = charting['1st'].apply(parse_sequence)

charting['2nd'] = charting['2nd'].astype(str)
charting['2nd_test'] = charting['2nd'].apply(parse_sequence)

In [13]:
charting[['match_id','1st_test', '2nd_test']].head()

,match_id,1st_test,2nd_test
0,20230908-M-US_Open-SF-Novak_Djokovic_-Ben_Shelton,2: 6 f2n #,0: nan
1,20230908-M-US_Open-SF-Novak_Djokovic_-Ben_Shelton,0: 4n,8: 6 b19 f1 b2 s1 f3 f2 j2 *
2,20230908-M-US_Open-SF-Novak_Djokovic_-Ben_Shelton,0: 4d,4: 4 b28 f2 o1 *
3,20230908-M-US_Open-SF-Novak_Djokovic_-Ben_Shelton,3: 6 s28 f3 *,0: nan
4,20230908-M-US_Open-SF-Novak_Djokovic_-Ben_Shelton,3: 4 b37 b3 *,0: nan


### Verificacion de la cantidad de golpes
A modo de verificacion, se comparará la cantidad de golpes obtenida mediante el parsing con la cantidad de golpes que se encuentra en la columna 'rallyCount'.
De esta manera, corroboramos que la información no tenga errores, ni de parsing ni de data entry.

In [14]:
def get_rally_count(row):
    
    """Get the rally count of parsed shots"""
    
    if row['1stIn'] == 1:
        if row['1st_test'][-1] != '*':
            return int(row['1st_test'].split(':')[0]) - 1
        else:
            return int(row['1st_test'].split(':')[0])
    else:
        if row['2nd_test'][-1] != '*':
            return int(row['2nd_test'].split(':')[0]) - 1
        else:
            return int(row['2nd_test'].split(':')[0])

In [15]:
charting.loc[:, 'rallyCountVerification'] = charting.apply(get_rally_count, axis=1)
charting.loc[:, 'rallyCountVerification'] = charting['rallyCountVerification'].apply(lambda x: x if x != -1 else 0)

In [16]:
charting = charting.dropna(subset=['rallyCount'])  # Drop rows with NaN in 'rallyCount'
charting[charting.columns[charting.columns.get_loc('rallyCount')]] = charting['rallyCount'].astype(int)

In [22]:
# De esta manera se puede ver cuantos valores son diferentes, comprobando que la diferencia se da por errores de data entry
differences = charting[charting['rallyCountVerification'].ne(charting['rallyCount'])]
print(f'Cantidad de valores diferentes: {differences.shape[0]}')

differences[['rallyCount', '1st_test', '1st','2nd_test','2nd', 'rallyCountVerification']].sample(5)

Cantidad de valores diferentes: 68417


,rallyCount,1st_test,1st,2nd_test,2nd,rallyCountVerification
244509,7,0: 4w,4w,7: 5 f28 b2 f3 b2 b1 f3 v#,5f28b2f3b2b1f3v#,6
209976,1,1: 6 bw#,6bw#,0: nan,nan,0
132520,1,1: 4 f#,4f#,0: nan,nan,0
195264,3,3: 4 f28 z3 s#,4f28z3s#,0: nan,nan,2
222193,6,0: 5n,5n,1: 5 fbfff*,5fbfff*,1


De esta manera eliminamos los errores de data entry

Finalmente, se procede a limpiar las columnas '1st_test' y '2nd_test' para que queden solo los golpes.

In [18]:
charting.loc[:,'1st_test'] = charting['1st_test'].str.split(':').str[1]
charting.loc[:,'2nd_test'] = charting['2nd_test'].str.split(':').str[1]

In [19]:
charting.loc[:, '1st_test'] = charting['1st_test'].str.strip()
charting.loc[:, '2nd_test'] = charting['2nd_test'].str.strip()

## Golpes por jugador
Para identificar los golpes de cada jugador, se procede a dividir la secuencia de golpes en dos. También se identifica el jugador que saca y el que recibe.
Esta información esta presente en la columna 'match_id', pero no viene en un formato amigable.

In [20]:
point_by_player = charting[['match_id', 'Pt', '1st_test', '2nd_test']]
split_data = point_by_player['match_id'].str.split('-').str[-2:]
point_by_player[['Player_svr', 'Player_ret']] = pd.DataFrame(split_data.to_list(), index=point_by_player.index)
point_by_player[['Player_svr', 'Player_ret']] = point_by_player[['Player_svr', 'Player_ret']].apply(lambda x: x.str.replace('_', ' '))
#strip() para quitar espacios en blanco
point_by_player[['Player_svr', 'Player_ret']] = point_by_player[['Player_svr', 'Player_ret']].apply(lambda x: x.str.strip()) 
point_by_player.head()

,match_id,Pt,1st_test,2nd_test,Player_svr,Player_ret
0,20230908-M-US_Open-SF-Novak_Djokovic_-Ben_Shelton,1,6 f2n #,nan,Novak Djokovic,Ben Shelton
1,20230908-M-US_Open-SF-Novak_Djokovic_-Ben_Shelton,2,4n,6 b19 f1 b2 s1 f3 f2 j2 *,Novak Djokovic,Ben Shelton
2,20230908-M-US_Open-SF-Novak_Djokovic_-Ben_Shelton,3,4d,4 b28 f2 o1 *,Novak Djokovic,Ben Shelton
3,20230908-M-US_Open-SF-Novak_Djokovic_-Ben_Shelton,4,6 s28 f3 *,nan,Novak Djokovic,Ben Shelton
4,20230908-M-US_Open-SF-Novak_Djokovic_-Ben_Shelton,5,4 b37 b3 *,nan,Novak Djokovic,Ben Shelton


In [21]:
def merge_last_two_strings(lst):
    
    """Concatena los ultimos dos strings de una lista"""
    
    if len(lst) >= 2: # Solo si hay dos o mas elementos
        # Concatena
        merged_value = lst[-2] + lst[-1]
        # Reemplaza los dos ultimos valores por el valor concatenado
        return lst[:-2] + [merged_value]
    else:
        return lst

In [22]:
point_by_player['1st_test'] = point_by_player['1st_test'].str.split(" ").apply(merge_last_two_strings)
point_by_player['2nd_test'] = point_by_player['2nd_test'].str.split(" ").apply(merge_last_two_strings)

def split_by_index(lst):
    
    """Divide una lista en dos, segun el indice del elemento. 
    De esta manera, se divide entre golpes del sacador y golpes del receptor."""
    
    even_index_values = [lst[i] for i in range(len(lst)) if i % 2 == 0]  # Valores en indices pares
    odd_index_values = [lst[i] for i in range(len(lst)) if i % 2 != 0]   # Valores en indices impares
    return even_index_values, odd_index_values

# Aplico la funcion y creo las columnas.
point_by_player[['1st_player_svr', '1st_player_ret']] = pd.DataFrame(point_by_player['1st_test'].apply(split_by_index).tolist(), index=point_by_player.index)
point_by_player[['2nd_player_svr', '2nd_player_ret']] = pd.DataFrame(point_by_player['2nd_test'].apply(split_by_index).tolist(), index=point_by_player.index)

# Convierto las listas en strings
point_by_player['1st_player_svr'] = point_by_player['1st_player_svr'].apply(lambda x: ' '.join(x))
point_by_player['1st_player_ret'] = point_by_player['1st_player_ret'].apply(lambda x: ' '.join(x))
point_by_player['2nd_player_svr'] = point_by_player['2nd_player_svr'].apply(lambda x: ' '.join(x))
point_by_player['2nd_player_ret'] = point_by_player['2nd_player_ret'].apply(lambda x: ' '.join(x))

In [23]:
#merge on match_id and Pt
point_by_player = point_by_player.drop(columns=['1st_test', '2nd_test'])
charting = pd.merge(charting, point_by_player, on=['match_id', 'Pt'])

In [24]:
def swap_columns(row):
    
    """Intercambia las columnas de sacador y receptor, si el sacador es el jugador 2"""
    
    if row['Svr'] == 2:
        row['Player_svr'], row['Player_ret'] = row['Player_ret'], row['Player_svr']
    return row

charting = charting.apply(swap_columns, axis=1)

# Filtro de datos: rallyCountVerification

In [25]:
# Finalmente, se filtran los datos que no tienen errores en la columna 'rallyCountVerification'. Evitamos sobretodo los errores de data entry.
charting_limpio = charting[charting['rallyCountVerification'].eq(charting['rallyCount'])]
charting_limpio.head()

,match_id,Pt,Set1,Set2,Gm1,Gm2,Pts,Gm#,TbSet,TB?,...,rallyCount,1st_test,2nd_test,rallyCountVerification,Player_svr,Player_ret,1st_player_svr,1st_player_ret,2nd_player_svr,2nd_player_ret
0,20230908-M-US_Open-SF-Novak_Djokovic_-Ben_Shelton,1,0,0,0.0,0.0,0-0,1 (1),1,0,...,1,6 f2n #,nan,1,Novak Djokovic,Ben Shelton,6,f2n#,nan,
1,20230908-M-US_Open-SF-Novak_Djokovic_-Ben_Shelton,2,0,0,0.0,0.0,15-0,1 (2),1,0,...,8,4n,6 b19 f1 b2 s1 f3 f2 j2 *,8,Novak Djokovic,Ben Shelton,4n,,6 f1 s1 f2,b19 b2 f3 j2*
2,20230908-M-US_Open-SF-Novak_Djokovic_-Ben_Shelton,3,0,0,0.0,0.0,15-15,1 (3),1,0,...,4,4d,4 b28 f2 o1 *,4,Novak Djokovic,Ben Shelton,4d,,4 f2,b28 o1*
3,20230908-M-US_Open-SF-Novak_Djokovic_-Ben_Shelton,4,0,0,0.0,0.0,15-30,1 (4),1,0,...,3,6 s28 f3 *,nan,3,Novak Djokovic,Ben Shelton,6 f3*,s28,nan,
4,20230908-M-US_Open-SF-Novak_Djokovic_-Ben_Shelton,5,0,0,0.0,0.0,30-30,1 (5),1,0,...,3,4 b37 b3 *,nan,3,Novak Djokovic,Ben Shelton,4 b3*,b37,nan,


In [26]:
print(charting_limpio.shape)
print(charting.shape)

(741257, 39)
(810051, 39)


# EDA

In [27]:
#merge last two in 1st_test
charting_limpio['1st_final'] = charting_limpio['1st_test'].str.split(" ").apply(merge_last_two_strings)
charting_limpio['1st_final'] = charting_limpio['1st_final'].apply(lambda x: ' '.join(x))

charting_limpio['2nd_final'] = charting_limpio['2nd_test'].str.split(" ").apply(merge_last_two_strings)
charting_limpio['2nd_final'] = charting_limpio['2nd_final'].apply(lambda x: ' '.join(x))

In [28]:
from collections import Counter
charting_limpio.to_csv('data_output/charting_clean.csv', index=False)

In [41]:
charting_limpio

,match_id,Pt,Set1,Set2,Gm1,Gm2,Pts,Gm#,TbSet,TB?,...,rallyCount,1st_test,2nd_test,rallyCountVerification,Player_svr,Player_ret,1st_player_svr,1st_player_ret,2nd_player_svr,2nd_player_ret
0,20230908-M-US_Open-SF-Novak_Djokovic_-Ben_Shelton,1,0,0,0.0,0.0,0-0,1 (1),1,0,...,1,6 f2n #,nan,1,Novak Djokovic,Ben Shelton,6,f2n#,nan,
1,20230908-M-US_Open-SF-Novak_Djokovic_-Ben_Shelton,2,0,0,0.0,0.0,15-0,1 (2),1,0,...,8,4n,6 b19 f1 b2 s1 f3 f2 j2 *,8,Novak Djokovic,Ben Shelton,4n,,6 f1 s1 f2,b19 b2 f3 j2*
2,20230908-M-US_Open-SF-Novak_Djokovic_-Ben_Shelton,3,0,0,0.0,0.0,15-15,1 (3),1,0,...,4,4d,4 b28 f2 o1 *,4,Novak Djokovic,Ben Shelton,4d,,4 f2,b28 o1*
3,20230908-M-US_Open-SF-Novak_Djokovic_-Ben_Shelton,4,0,0,0.0,0.0,15-30,1 (4),1,0,...,3,6 s28 f3 *,nan,3,Novak Djokovic,Ben Shelton,6 f3*,s28,nan,
4,20230908-M-US_Open-SF-Novak_Djokovic_-Ben_Shelton,5,0,0,0.0,0.0,30-30,1 (5),1,0,...,3,4 b37 b3 *,nan,3,Novak Djokovic,Ben Shelton,4 b3*,b37,nan,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
809969,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,170,1,1,5.0,4.0,40-40,28 (9),1,0,...,2,6 b38 z2d @,nan,2,Luis Ayala,Nicola Pietrangeli,6 z2d@,b38,nan,
809970,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,171,1,1,5.0,4.0,40-AD,28 (10),1,0,...,3,4n,4 s28 z1 *,3,Luis Ayala,Nicola Pietrangeli,4n,,4 z1*,s28
809972,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,173,1,1,5.0,4.0,40-AD,28 (12),1,0,...,3,4 s38 z1 *,nan,3,Luis Ayala,Nicola Pietrangeli,4 z1*,s38,nan,
809973,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,174,1,1,5.0,4.0,40-40,28 (13),1,0,...,2,4 f27 i3n #,nan,2,Luis Ayala,Nicola Pietrangeli,4 i3n#,f27,nan,


In [42]:
test = charting_limpio.copy()

test = test[['Player_svr', '1st_player_svr', '2nd_player_svr']] 

test['1st_player_svr'] = test['1st_player_svr'].str.split(' ')

#explode both columns
exploded = test.copy()
exploded = test.explode('1st_player_svr')

exploded.groupby('Player_svr')['1st_player_svr'].value_counts(normalize=False)

Player_svr        1st_player_svr
Aaron Krickstein  f1                208
                  4                 177
                  6                 173
                  b3                171
                  f3                145
                                   ... 
Zsombor Piros     r1                  1
                  r3n#                1
                  s3                  1
                  s3n@                1
                  u37*                1
Name: 1st_player_svr, Length: 65677, dtype: int64

In [43]:
exploded[exploded['Player_svr'] == 'Novak Djokovic']['1st_player_svr'].value_counts(normalize=False)

4        8113
6        6869
f1       5707
f3       5440
b3       5038
         ... 
z27*        1
u27*        1
b17*        1
b18w#       1
j3w@        1
Name: 1st_player_svr, Length: 394, dtype: int64